# Notebook 02: Building a Simple Agent with Tools

## 🎯 What is This Notebook About?

In Notebook 01, we learned about autonomous agents conceptually. Now we're going to **build a real agent** that can actually reason and take actions!

**What we'll do:**
1. **Set up llamastack** - Connect to the llamastack server for agent capabilities
2. **Create an agent** - Build an agent with tools using llamastack API
3. **Test the agent** - See the agent reason and take actions autonomously
4. **Understand the agent loop** - See how Observe → Think → Act works in practice

**Why this matters:**
- This is where theory becomes practice
- You'll see how an agent actually makes decisions
- You'll understand how tools enable agents to take actions
- This is the foundation for more advanced agents

---

## 📚 Key Concepts Explained

### What is LlamaStack?

**LlamaStack** is a unified framework for building AI applications. It provides:
- **Agent APIs** - For creating and managing autonomous agents
- **Tool Integration** - Connect tools (actions) to agents
- **LLM Management** - Handle different LLM providers (like Ollama)
- **Unified Interface** - One API for all agent operations

**Think of it like:** A platform that connects AI reasoning (LLM) with actions (tools).

### Agent Creation Process

When we create an agent, we need to provide:
1. **Instructions** - What the agent should do and how it should behave
2. **Tools** - What actions the agent can take
3. **Model** - Which LLM to use for reasoning (e.g., Ollama with llama3.2:3b)

### The Agent Execution Flow

```
1. User gives task → "Check all services and fix any problems"
2. Agent receives task
3. Agent uses LLM to reason about what to do
4. Agent selects appropriate tool(s)
5. Agent executes tool(s)
6. Agent evaluates results
7. Agent continues or completes task
```

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Know how to set up and connect to llamastack
- ✅ Understand how to create an agent with tools
- ✅ See how agents reason and select tools
- ✅ Understand the agent execution flow
- ✅ Be ready to build more complex agents

---

## ⚠️ Prerequisites

**IMPORTANT:** LlamaStack is **REQUIRED** for this notebook. The agent will not work without it.

Before starting, make sure:
- ✅ Ollama is running (`ollama serve`)
- ✅ LlamaStack server is running (see `scripts/start_llama_stack.py`)
- ✅ Model `llama3.2:3b` is available (`ollama pull llama3.2:3b`)

**If llamastack is not available, the notebook will fail with an error.**



In [ ]:
# Import required libraries
import sys
from pathlib import Path
import os
import requests
import json

# Add src directory to path
src_path = Path("../src").resolve()
sys.path.insert(0, str(src_path))

# Import our agent framework components
from environment import SimulatedEnvironment
from tools import ToolRegistry
from agent import AutonomousAgent
from memory import AgentMemory

print("✅ Libraries imported successfully!")
print("🤖 Ready to build an autonomous agent!")


## 1. Check LlamaStack Connection

**What we're doing:** Verifying that llamastack server is available.

**Why:** The agent **requires** llamastack to function. Without it, the agent cannot be created or execute tasks.

**What to expect:**
- If llamastack is running, we'll see a success message
- If not, we'll see an error and the notebook will stop (llamastack is required)


In [ ]:
# Check if llamastack server is available
# LlamaStack is REQUIRED for the agent to work
llamastack_url = os.getenv("LLAMA_STACK_URL", "http://localhost:8321")

try:
    # Use /v1/health endpoint to verify llamastack is running
    response = requests.get(f"{llamastack_url}/v1/health", timeout=5)
    response.raise_for_status()
    health_data = response.json()
    print("✅ LlamaStack server is running!")
    print(f"   URL: {llamastack_url}")
    print(f"   Status: {health_data.get('status', 'OK')}")
    llamastack_available = True
except requests.exceptions.RequestException as e:
    print("❌ ERROR: LlamaStack server is not available")
    print(f"   Error: {e}")
    print("\n⚠️  LlamaStack is REQUIRED for this notebook to work.")
    print("   Please start llamastack before continuing:")
    print("   python scripts/start_llama_stack.py")
    print("\n   Or set LLAMA_STACK_URL environment variable if running on a different host/port.")
    raise ConnectionError(
        f"Cannot connect to llamastack server at {llamastack_url}. "
        "Please ensure llamastack is running before continuing."
    )


## 2. Set Up Environment and Tools

**What we're doing:** Creating the simulated IT environment and tool registry.

**Why:** The agent needs an environment to interact with and tools to take actions.

**What to expect:**
- A simulated environment with IT services
- A registry of available tools
- Tools ready for the agent to use


In [ ]:
# Create simulated IT environment
services = [
    "web-server",
    "database",
    "cache-service",
    "api-gateway",
    "monitoring-service"
]

env = SimulatedEnvironment(initial_services=services)
print("✅ Simulated environment created!")
print(f"   Services: {', '.join(services)}")

# Create tool registry
tool_registry = ToolRegistry(environment=env)
print(f"\n✅ Tool registry created!")
print(f"   Available tools: {len(tool_registry.list_tools())}")

# Show available tools
print("\n🛠️  Available Tools:")
for tool in tool_registry.list_tools():
    print(f"   - {tool['name']}")


## 3. Create an Autonomous Agent

**What we're doing:** Creating an autonomous agent using llamastack.

**Why:** This is where we bring everything together - environment, tools, and AI reasoning.

**What to expect:**
- An agent will be created in llamastack
- The agent will have access to all our tools
- The agent will be ready to receive tasks

**How it works:**
- The agent uses llamastack API to create an agent instance
- Tools are registered with the agent
- The agent gets instructions on how to behave
- The agent is ready to reason and act


In [ ]:
# Create agent memory (for learning)
memory = AgentMemory()

# Create the autonomous agent
# This will fail if llamastack is not available
print("🤖 Creating autonomous agent in llamastack...")
print("-" * 60)

try:
    agent = AutonomousAgent(
        tool_registry=tool_registry,
        memory=memory,
        llamastack_url=llamastack_url,
        verbose=True
    )
    
    print(f"\n✅ Agent created successfully!")
    print(f"   Agent ID: {agent.agent_id}")
    print(f"   LlamaStack URL: {llamastack_url}")
    print(f"   Tools available: {len(tool_registry.list_tools())}")
    print(f"   Memory enabled: {memory is not None}")
except ConnectionError as e:
    print(f"\n❌ Failed to create agent: {e}")
    print("\n⚠️  Please ensure llamastack is running and try again.")
    raise
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    print("\n⚠️  Please check the error message above and ensure llamastack is properly configured.")
    raise


## 4. Test the Agent: Simple Task

**What we're doing:** Giving the agent a simple task and watching it work.

**Why:** This demonstrates the agent's reasoning and action-taking capabilities.

**What to expect:**
- The agent will receive a task
- The agent will reason about what to do
- The agent will select and use appropriate tools
- We'll see the results of the agent's actions

**Task:** "Check the status of all services"

This is a simple read-only task that lets us see the agent in action without making changes.


In [ ]:
# Test 1: Simple observation task
print("📋 Task 1: Check status of all services")
print("=" * 60)

task = "Check the status of all services in the environment and report their current state."

result = agent.run(task)

print("\n📊 Agent Result:")
print("-" * 60)
if result["success"]:
    print(result["result"])
    if "tool_calls" in result and result["tool_calls"]:
        print(f"\n🔧 Tools used: {len(result['tool_calls'])}")
        for i, tool_call in enumerate(result.get("tool_calls", []), 1):
            tool_name = tool_call.get("name", "unknown")
            print(f"   {i}. {tool_name}")
else:
    print(f"❌ Error: {result.get('error', 'Unknown error')}")

print("\n" + "=" * 60)


## 5. Test the Agent: Problem Detection and Remediation

**What we're doing:** Creating a problem and asking the agent to fix it.

**Why:** This shows how agents can detect problems and take corrective actions.

**What to expect:**
- We'll simulate a service failure
- The agent will detect the problem
- The agent will decide how to fix it
- The agent will take action to remediate

**Task:** "Analyze the environment and fix any problems you find"

This is a more complex task that requires:
1. Observing the environment
2. Identifying problems
3. Deciding on a solution
4. Taking corrective action
5. Verifying the fix worked


In [ ]:
# Test 2: Problem detection and remediation
print("📋 Task 2: Detect and fix problems")
print("=" * 60)

# Simulate a problem: web-server is failed
print("\n⚠️  Simulating a problem: web-server has failed")
env.simulate_failure("web-server")

# Check status before agent intervention
print("\n📊 Status BEFORE agent intervention:")
status_before = tool_registry.execute_tool("check_service_status", service_name="web-server")
print(status_before)

# Give agent the task
print("\n🤖 Agent task: Analyze the environment and fix any problems you find")
print("-" * 60)

task = "Analyze the current state of all IT services. If you find any services that are failed or degraded, take appropriate actions to fix them. Verify that your actions were successful."

result = agent.run(task)

print("\n📊 Agent Result:")
print("-" * 60)
if result["success"]:
    print(result["result"])
    if "tool_calls" in result and result["tool_calls"]:
        print(f"\n🔧 Tools used ({len(result['tool_calls'])}):")
        for i, tool_call in enumerate(result["tool_calls"], 1):
            tool_name = tool_call.get("name", "unknown")
            print(f"   {i}. {tool_name}")
else:
    print(f"❌ Error: {result.get('error', 'Unknown error')}")

# Check status after agent intervention
print("\n📊 Status AFTER agent intervention:")
status_after = tool_registry.execute_tool("check_service_status", service_name="web-server")
print(status_after)

print("\n" + "=" * 60)


## 6. Understanding Agent Memory

**What we're doing:** Exploring what the agent has learned from its actions.

**Why:** Agents can remember past actions and learn from them, improving over time.

**What to expect:**
- We'll see statistics about actions taken
- We'll see what the agent remembers
- We'll understand how memory helps the agent improve

**Key Insight:** Memory allows agents to:
- Avoid repeating failed actions
- Reuse successful solutions
- Learn patterns from experience


In [ ]:
# Check agent memory
print("📚 Agent Memory Statistics")
print("=" * 60)

memory_stats = agent.get_memory_stats()

print("\n📊 Action Statistics:")
stats = memory_stats.get("action_statistics", {})
print(f"   Total actions: {stats.get('total', 0)}")
print(f"   Successful: {stats.get('successful', 0)}")
print(f"   Failed: {stats.get('failed', 0)}")
print(f"   Success rate: {stats.get('success_rate', 0.0):.1%}")

print("\n📝 Recent Actions:")
recent = memory_stats.get("recent_actions", [])
if recent:
    for i, action in enumerate(recent, 1):
        status = "✅" if action["success"] else "❌"
        print(f"   {i}. {status} {action['type']} (at {action['timestamp']:.1f})")
else:
    print("   No actions recorded yet")

print("\n🎯 Problems Solved:")
print(f"   Total: {memory_stats.get('total_problems_solved', 0)}")

print("\n" + "=" * 60)


## 7. Advanced Task: Multi-Step Problem Solving

**What we're doing:** Giving the agent a complex task that requires multiple steps.

**Why:** Real-world problems often require multiple actions in sequence.

**What to expect:**
- The agent will break down the problem into steps
- The agent will execute multiple tools in sequence
- The agent will verify results between steps
- We'll see the agent's reasoning process

**Task:** "The database service is experiencing high load. Investigate and fix it."

This requires:
1. Checking the database status
2. Identifying the problem (high CPU/memory)
3. Deciding on a solution (scale vs restart)
4. Executing the solution
5. Verifying it worked


In [ ]:
# Test 3: Multi-step problem solving
print("📋 Task 3: Multi-step problem solving")
print("=" * 60)

# Reset environment and create a complex problem
env.reset()
env.simulate_degradation("database")  # High CPU/memory

print("\n⚠️  Problem: Database service is experiencing high load")
initial_status = tool_registry.execute_tool("check_service_status", service_name="database")
print(f"   Initial status: {initial_status}")

# Give agent complex task
print("\n🤖 Agent task: Investigate and fix the database performance issue")
print("-" * 60)

task = """The database service is experiencing high load and performance issues. 
Please investigate the current state of the database service, identify the root cause, 
and take appropriate corrective action. After taking action, verify that the problem has been resolved."""

result = agent.run(task)

print("\n📊 Agent Result:")
print("-" * 60)
if result["success"]:
    print(result["result"])
    
    # Show tool calls if available
    if "tool_calls" in result and result["tool_calls"]:
        print(f"\n🔧 Tools used ({len(result['tool_calls'])}):")
        for i, tool_call in enumerate(result["tool_calls"], 1):
            tool_name = tool_call.get("name", "unknown")
            print(f"   {i}. {tool_name}")
else:
    print(f"❌ Error: {result.get('error', 'Unknown error')}")

# Verify final state
print("\n📊 Final Status:")
final_status = tool_registry.execute_tool("check_service_status", service_name="database")
print(final_status)

print("\n" + "=" * 60)


## 8. Key Takeaways

**What we learned:**

1. **Agent Creation** - How to create an agent with tools using llamastack
2. **Agent Execution** - How agents receive tasks and execute them
3. **Tool Selection** - How agents choose which tools to use
4. **Multi-Step Reasoning** - How agents break down complex problems
5. **Memory** - How agents remember and learn from actions

**Key Insights:**

- **Agents reason before acting** - They analyze the situation first
- **Agents can use multiple tools** - They chain actions together
- **Agents verify results** - They check if actions worked
- **Agents learn from experience** - Memory helps them improve

**What's different from automation:**

- **Automation:** "If X, then Y" (fixed rules)
- **Agent:** "I see X. Let me think about the best way to handle it. I'll try Y and verify it works." (reasoning)

---

## 🎓 Summary

**Building an agent involves:**

1. **Environment** - Where the agent operates (simulated IT infrastructure)
2. **Tools** - Actions the agent can take (check status, restart, scale)
3. **LLM** - Provides reasoning capability (via llamastack)
4. **Memory** - Stores experiences for learning

**The agent workflow:**

```
Task → Agent → Reasoning (LLM) → Tool Selection → Action → Verification → Learning
```

**What's next:**

In Notebook 03, we'll explore:
- More sophisticated decision-making
- Handling failures and retries
- Safety checks and validation
- Advanced agent patterns

---

**Congratulations!** You've built your first autonomous agent! 🎉
